In [0]:
dbutils.library.installPyPI("mlflow")
#dbutils.library.restartPython()

Out[1]: True

In [0]:
from pyspark.sql.types import StructType
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import *      # for window() function
from typing import List
from pyspark.sql.types import *
import pandas as pd
import time
from datetime import datetime, timedelta, timezone
import mlflow
from pyspark.sql.types import StructType
import dateutil.parser
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.types import StructType

version = "job_v-4.7"

In [0]:
%sql
drop table raw_log_data_delta_PN_job;
drop table anomalies_data_delta_PN_job;

In [0]:
%sql
--CREATE TABLE IF NOT EXISTS raw_log_data_delta_PN_job (
--  account_id STRING,
--  agent_id STRING,
--  event STRING, 
--  timestamp TIMESTAMP
-- )
--USING DELTA;
CREATE TABLE IF NOT EXISTS raw_log_data_delta_PN_job (
  event STRING
 )
USING DELTA;


CREATE TABLE IF NOT EXISTS anomalies_data_delta_PN_job (
  user_id STRING,
  Ips LONG,
  prediction DOUBLE
)
USING DELTA;


In [0]:
#57c7413abca837e974000009
inputPath = "dbfs:/mnt/kafka_raw/57c7413abca837e974000009/"

#reads From Kafka
eventsDF = (spark.readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "td-guardian.tdx.sandcitadel.com:9093")
  .option("subscribe", "event-splitter.audit_logs")
  .load())

#getting the event only
eventsDF = (
            eventsDF
            .selectExpr("CAST(value AS STRING)")
            .select( col("value").alias("event"))
           )




In [0]:
display(eventsDF)

event
"{""timestamp"":""2020-10-20T11:23:39.989084Z"",""account_id"": ""account_259"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_6676"",""ip_address"": [""user_6676_ip_1""]}}}"
"{""timestamp"":""2020-10-20T11:23:39.989122Z"",""account_id"": ""account_529"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_43827"",""ip_address"": [""user_43827_ip_2""]}}}"
"{""timestamp"":""2020-10-20T11:23:39.990230Z"",""account_id"": ""account_454"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_12389"",""ip_address"": [""user_12389_ip_0""]}}}"
"{""timestamp"":""2020-10-20T11:23:39.991359Z"",""account_id"": ""account_468"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_16250"",""ip_address"": [""user_16250_ip_2""]}}}"
"{""timestamp"":""2020-10-20T11:23:39.992494Z"",""account_id"": ""account_97"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_85926"",""ip_address"": [""user_85926_ip_0""]}}}"
"{""timestamp"":""2020-10-20T11:23:39.993628Z"",""account_id"": ""account_760"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_55684"",""ip_address"": [""user_55684_ip_2""]}}}"
"{""timestamp"":""2020-10-20T11:23:39.994750Z"",""account_id"": ""account_40"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_71398"",""ip_address"": [""user_71398_ip_2""]}}}"
"{""timestamp"":""2020-10-20T11:23:39.995869Z"",""account_id"": ""account_179"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_18125"",""ip_address"": [""user_18125_ip_0""]}}}"
"{""timestamp"":""2020-10-20T11:23:39.996992Z"",""account_id"": ""account_832"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_19242"",""ip_address"": [""user_19242_ip_0""]}}}"
"{""timestamp"":""2020-10-20T11:23:39.997033Z"",""account_id"": ""account_538"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_82988"",""ip_address"": [""user_82988_ip_0""]}}}"


In [0]:
#writes to Raw table
(eventsDF.writeStream.queryName("write_raw_table")
  .outputMode("append")
  .option("checkpointLocation", "/mnt/delta/events/_checkpoints/etl-from-json_PN_"+version)
  .table("raw_log_data_delta_PN_job")
)



Out[5]: <pyspark.sql.streaming.StreamingQuery at 0x7f83f8411690>

In [0]:
#We may need to user OPTIMIZE, which deals with small files, merge them and compact them into larger files
raw_data = spark.readStream.format("delta").table("raw_log_data_delta_PN_job")


event
"{""timestamp"":""2020-10-15T17:43:09.925134Z"",""account_id"": ""account_77"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_6416"",""ip_address"": [""user_6416_ip_0""]}}}"
"{""timestamp"":""2020-10-15T17:43:09.926251Z"",""account_id"": ""account_51"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_35690"",""ip_address"": [""user_35690_ip_2""]}}}"
"{""timestamp"":""2020-10-15T17:43:09.927379Z"",""account_id"": ""account_365"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_22491"",""ip_address"": [""user_22491_ip_2""]}}}"
"{""timestamp"":""2020-10-15T17:43:09.928506Z"",""account_id"": ""account_423"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_6747"",""ip_address"": [""user_6747_ip_0""]}}}"
"{""timestamp"":""2020-10-15T17:43:09.929628Z"",""account_id"": ""account_62"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_52067"",""ip_address"": [""user_52067_ip_0""]}}}"
"{""timestamp"":""2020-10-15T17:43:09.930756Z"",""account_id"": ""account_683"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_54248"",""ip_address"": [""user_54248_ip_2""]}}}"
"{""timestamp"":""2020-10-15T17:43:09.931873Z"",""account_id"": ""account_513"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_40031"",""ip_address"": [""user_40031_ip_1""]}}}"
"{""timestamp"":""2020-10-15T17:43:09.932993Z"",""account_id"": ""account_62"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_89358"",""ip_address"": [""user_89358_ip_0""]}}}"
"{""timestamp"":""2020-10-15T17:43:09.933038Z"",""account_id"": ""account_486"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_2885"",""ip_address"": [""user_2885_ip_2""]}}}"
"{""timestamp"":""2020-10-15T17:43:09.934239Z"",""account_id"": ""account_201"",""event_type"": ""create_audit_operation"",""event"": {""audit"": {""operation"": ""user_session_created""},""actor"": {""user_id"":""user_34756"",""ip_address"": [""user_34756_ip_1""]}}}"


In [0]:
fullschema = (  StructType()
  .add("timestamp","timestamp")                
  .add("account_id","string")
  .add("event_type","string")
  .add("event",StructType()
         .add("audit",StructType()
              .add("operation","string")
         )
         .add("actor",StructType()
             .add("user_id","string")
             .add("ip_address",ArrayType(StringType()))
         ) 
      )
)


In [0]:
run_id = "09840597c6e04f279aaa27be313c6e73"
model_uri = "runs:/" + run_id + "/sklearn-model"
model = mlflow.pyfunc.spark_udf(spark, model_uri)


filtered_data = (eventsDF
                  .select( from_json("event", fullschema).alias("data"))
                  .select( col("data.timestamp").alias("timestamp") ,   col("data.event.actor.user_id").alias("user_id"),  col("data.event.actor.ip_address").alias("ip_address") )   
                  .withColumn("timestamp",to_timestamp(to_date("timestamp","yyyy-MM-dd"),"yyyy-MM-dd"))
                  #.where(col("timestamp") >= datetime.now().astimezone(timezone.utc).strftime("%Y-%m-%dT00:00:00.000+0000") )
                  .withWatermark("timestamp", "5 seconds")
                  .groupBy("timestamp","user_id").agg(approx_count_distinct('ip_address').alias('Ips'))
                  #.groupBy(window("timestamp", "5 minute").alias("timestamp"),"user_id").agg(approx_count_distinct('ip_address').alias('Ips'))
                  
                  .select("user_id","Ips")
                  .withColumn("prediction", model("Ips"))  
                )




In [0]:
display(filtered_data)

user_id,Ips,prediction
user_84873,3,1.0
user_95770,3,1.0
user_76225,3,1.0
user_84610,3,1.0
user_99266,3,1.0
user_39950,3,1.0
user_64561,3,1.0
user_10661,3,1.0
user_98763,3,1.0
user_23450,3,1.0


In [0]:
run_id = "09840597c6e04f279aaa27be313c6e73"
model_uri = "runs:/" + run_id + "/sklearn-model"
model = mlflow.pyfunc.spark_udf(spark, model_uri)


filtered_data = (raw_data
                  .select( from_json("event", fullschema).alias("data"))
                  .select( col("data.timestamp").alias("timestamp") ,   col("data.event.actor.user_id").alias("user_id"),  col("data.event.actor.ip_address").alias("ip_address") )   
                  #.withColumn("timestamp",to_timestamp(to_date("timestamp","yyyy-MM-dd"),"yyyy-MM-dd"))
                  #.where(col("timestamp") >= datetime.now().astimezone(timezone.utc).strftime("%Y-%m-%dT00:00:00.000+0000") )
                  
                  #.groupBy(date_trunc("day","timestamp").alias("timestamp"),"user_id").agg(approx_count_distinct('ip_address').alias('Ips'))
                  .groupBy(window("timestamp", "5 minute").alias("timestamp"),"user_id").agg(approx_count_distinct('ip_address').alias('Ips'))
                  .withWatermark("timestamp", "5 seconds")
                  #.select("user_id","Ips")
                  #.withColumn("prediction", model("Ips"))  
                )

In [0]:
display(filtered_data)

In [0]:
#write data to anomalies' table
(filtered_data.writeStream.queryName("write_anomalies_table")
  .format("delta") 
  .outputMode("append")
  .option("checkpointLocation", "/mnt/delta/events/_checkpoints/anomalies_"+version)
  .table("anomalies_data_delta_PN_job")
)



Out[17]: <pyspark.sql.streaming.StreamingQuery at 0x7fee0ddafe10>

In [0]:
%sql
select count(*) from anomalies_data_delta_PN_job

count(1)
0


In [0]:
%sql
drop table anomalies_data_delta_PN_job;

CREATE TABLE IF NOT EXISTS anomalies_data_delta_PN_job (
  user_id STRING,
  Ips LONG,
  prediction DOUBLE
)
USING DELTA;

In [0]:
%sql
select * from anomalies_data_delta_PN_job